In [1]:
import torch 
import torch.nn.functional as F

In [4]:
# read all the words from text 
words = open('names.txt','r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [5]:
# number of words 
len(words)

32033

In [11]:
# build the vocabulary of chars and mapping from/to integers 
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [17]:
# build the data set 

block_size = 3 # the context length (how many chars to consider to make prediction)
X, Y = [], []
for w in words[:3]:              # let's test the code for first 3 words 
    context = [0]*block_size     
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix] # crop and append (this moves the window by one char forward)
        
X = torch.tensor(X)
Y = torch.tensor(Y)

In [27]:
X[:6],Y[:6]

(tensor([[ 0,  0,  0],
         [ 0,  0,  5],
         [ 0,  5, 13],
         [ 5, 13, 13],
         [13, 13,  1],
         [ 0,  0,  0]]),
 tensor([ 5, 13, 13,  1,  0, 15]))

In [26]:
words[0]

'emma'

In [29]:
# build the data set 

block_size = 3 # the context length (how many chars to consider to make prediction)

def build_dataset(words):
    X, Y = [], []
    for w in words[:3]:              # let's test the code for first 3 words 
        context = [0]*block_size     
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append (this moves the window by one char forward)
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X,Y

# create training and test data sets 
import random
random.seed(42)
random.shuffle(words)
n1 = int(len(words)*0.8)
n2 = int(len(words)*0.9)

Xtr,Ytr = build_dataset(words[:n1])
Xdev,Ydev = build_dataset(words[n1:n2])
Xte,Yte = build_dataset(words[n2:])